In [ ]:
# |default_exp callback

In [ ]:
# |export
from functools import partial

from fastcore.foundation import L, Self
from fastcore.xtras import class2attr

# Callback
> Callbacks used by Learner and Collector to modify their behaviour

In [ ]:
# |export
class Callbackable:
    def __init__(self, cbs=None):
        self.cbs = L()
        self.add_cbs(cbs)

    def add_cbs(self, cbs):
        L(cbs).map(self.add_cb)
        return self

    def add_cb(self, cb):
        if isinstance(cb, (type, partial)):
            cb = cb()
        cb.parent = self
        setattr(self, cb.name, cb)
        self.cbs.append(cb)
        return self

    def __call__(self, event_name):
        for cb in self.cbs:
            event = getattr(cb, event_name, None)
            if callable(event):
                event()

    def cbs_state_dict(self):
        self.assert_unique()
        return {
            cb.name: cb.state_dict() for cb in self.cbs if hasattr(cb, "state_dict")
        }

    def load_cbs_state_dict(self, cbs_state_dict):
        self.assert_unique()
        for cb in self.cbs:
            if hasattr(cb, "load_state_dict") and cb.name in cbs_state_dict:
                cb.load_state_dict(cbs_state_dict[cb.name])

    def assert_unique(self):
        cb_names = self.cbs.map(Self.name)
        assert len(cb_names) == len(set(cb_names)), "Callback names must be unique"


class Callback:
    @property
    def name(self):
        return class2attr(self, "Callback")

In [ ]:
# |hide
# |eval: false
from nbdev.doclinks import nbdev_export

nbdev_export()